# Develop a Web chatbot using gradio and OpenAI's GPT-4o-mini

In [1]:
import gradio as gr
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from openai import OpenAI

In [2]:
load_dotenv(override=True)
openai = OpenAI()

## Read pdf files and use them as context for a chatbot using gradio and OpenAI's GPT-4o-mini

In [3]:
reader = PdfReader("../resume/resume_khangbuitranduy_updated.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [4]:
print(resume)

(+84)-815-594-665 |       khang.buitranduycse@hcmut.edu.vn |      khangbkk23 |       khangbkk23
KHANG BUI TRAN DUY
AI ENGINEER / DATA SCIENCE 
2023 - 2027
May 2025 - PresentI’m a highly motivated third-year Computer Science student with a strong foundation in Machine Learning, Data
Science, Computer Vision, and Natural Language Processing. I’m passionate about Artificial Intelligence,
continuously expanding my knowledge and applying it to real-world problems. I have hands-on experience with
TensorFlow, PyTorch, and Scikit-learn, along with solid skills in Machine Learning and Deep Learning, and growing
expertise in Reinforcement Learning. Beyond academics, I actively engage in social and community activities, which have
strengthened my teamwork, communication, and leadership skills. As an aspiring AI Engineer and Data Scientist, I am
eager to collaborate with experts, tackle challenging projects, and further develop my expertise with the goal of building
scalable, intelligent systems a

In [5]:
with open("../resume/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
print(summary)

I am Khang Bui Tran Duy, a highly motivated third-year Computer Science student at Ho Chi Minh City University of Technology with a CPA of 3.6/4.0. My expertise lies in Machine Learning, Deep Learning, and Reinforcement Learning, evidenced by my ongoing research at the Optimization AI Lab on Continual Learning strategies.Unlike a typical developer, I possess a strong 'Research-to-Production' mindset. I have hands-on experience building end-to-end pipelines—from data preprocessing and algorithm design (using PyTorch/TensorFlow) to model deployment (using Docker/FastAPI). I am eager to collaborate with experts to tackle challenging problems in Computer Vision and NLP, aiming to build scalable, intelligent systems that drive real-world impact.


In [7]:
name = "Duy Khang"

In [8]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{resume}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

### Build the chat function

In [9]:
model = "gpt-4o-mini"

def chat(message, history):
    messages = [{"role" : "system", "content" : system_prompt }] + history +[{"role" : "user", "content" : message}]
    response = openai.chat.completions.create(
        model = model,
        messages= messages
    )
    return response.choices[0].message.content



In [10]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Try to evaluate the answer of the chatbot by another Ollama model

In [21]:
from pydantic import BaseModel
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str
    score: int

In [13]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{resume}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

### Create a evaluation function

In [14]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [15]:
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama" 
)

deepseek_name = "deepseek-r1:7b"

In [ ]:
import json
import re

def extract_json_from_deepseek(text: str):
    if "</think>" in text:
        text = text.split("</think>")[-1]
    match = re.search(r'\{.*\}', text, re.DOTALL)
    
    if match:
        json_str = match.group(0)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            pass
    return None

In [ ]:
def evaluate(reply, message, history) -> Evaluation:
    print(f"Đang gửi yêu cầu tới Ollama ({deepseek_name})...")
    system_prompt = (
        "You are an AI Evaluator. "
        "You must output ONLY a valid JSON object. "
        "Do not include any text outside the JSON. "
        "The JSON must strictly follow this schema:\n"
        "{\n"
        '  "is_acceptable": true/false,\n'
        '  "feedback": "string evaluation",\n'
        '  "score": integer_0_to_10\n'
        "}"
    )

    user_content = evaluator_user_prompt(reply, message, history)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_content}
    ]

    try:
        response = client.chat.completions.create(
            model=deepseek_name,
            messages=messages,
            response_format={"type": "json_object"}, 
            temperature=0.1
        )
        
        raw_content = response.choices[0].message.content
        json_data = extract_json_from_deepseek(raw_content)

        if json_data:
            return Evaluation.model_validate(json_data)
        else:
            raise ValueError("Không tìm thấy JSON hợp lệ trong phản hồi.")

    except Exception as e:
        print(f"Lỗi parse: {e}")
        return Evaluation(
            is_acceptable=False,
            feedback=f"System Error: {str(e)}", 
            score=0
        )

In [23]:

messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model=model, messages=messages)
reply = response.choices[0].message.content

In [24]:

reply

'I currently do not hold a patent. My focus is on research and development in the fields of Machine Learning, Deep Learning, and Reinforcement Learning, particularly through my work in the Optimization AI Lab. While I strive to contribute to advancements in these areas and collaborate on innovative projects, I have not yet secured any patents. If you have further questions about my work or projects, feel free to ask!'

In [28]:
evaluate(reply, "do you hold a patent?", messages[:1])

Đang gửi yêu cầu tới Ollama (deepseek-r1:7b)...


Evaluation(is_acceptable=True, feedback='The assistant provided a clear and honest answer regarding the lack of current patents, while also highlighting their focus areas and willingness to discuss further.', score=9)

In [29]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=model, messages=messages)
    return response.choices[0].message.content

In [37]:

def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=model, messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [38]:

gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Đang gửi yêu cầu tới Ollama (deepseek-r1:7b)...
Passed evaluation - returning reply
Đang gửi yêu cầu tới Ollama (deepseek-r1:7b)...
Passed evaluation - returning reply
